In [3]:
!pip install transformers datasets torch

# Import necessary libraries
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments
from transformers import AutoTokenizer
from datasets import Dataset
import torch




In [4]:
data = {
    'text': [
        "Explanation: This is an apple. Label: Fruit",
        "Explanation: This is a dog. Label: Animal",
        "Explanation: This is a car. Label: Vehicle",
    ]
}

# Create a Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Load a pre-trained GPT-2 small model and tokenizer
# We use GPT-2 small for a quick example. For a real task, you'd likely use a larger model.
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
# Add a padding token to the tokenizer if it doesn't have one
# This is often necessary for batching in training
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=64) # Adjust max_length as needed

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare data for the language modeling task
# The labels are the input_ids themselves shifted by one position
tokenized_dataset = tokenized_dataset.map(
    lambda examples: {'labels': examples['input_ids']}, batched=True
)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    learning_rate=2e-5, # Can be Changed
    num_train_epochs=3,  # Small number of epochs for quick training # Can be changed
    per_device_train_batch_size=2, # Small batch size -- can be changed
    weight_decay=0.01,# Can also be changed
    logging_dir="./logs",
    logging_steps=10,
)

In [9]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset, # Use the full dataset as we don't have a separate eval set here
    eval_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

print("Training complete.")

# Inference Task
# You can now use the trained model for inference
# For example, to generate text based on an explanation:
# prompt = "Explanation: This is a building. Label:"
# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(inputs.input_ids, max_length=50)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# GLUE Benchmarks

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lathersanju97 (lathersanju97-sharda-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Training complete.
